This Notebook works as a backend for iterating on the agent for our hackathon.

To use,
1. Make a copy of it for yourself.
2. Add your own OpenAI Api key in secrets
3. Add the shared Google api key in secrets

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install openai langchain openapi-pydantic faiss-cpu tiktoken
!pip install py-near
!pip install base58
!pip install loguru

In [ ]:
import os
import openai

from google.colab import userdata
openai.api_key = userdata.get('openai_api_key')
os.environ['OPENAI_API_KEY'] = userdata.get('openai_api_key')

In [ ]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.embeddings import OpenAIEmbeddings

from langchain.prompts import PromptTemplate
from langchain.utilities import GoogleSearchAPIWrapper

from py_near.account import Account
import asyncio
from py_near.dapps.core import NEAR

In [ ]:
# Retrieval Augmented Generation, let's teach it some things
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool
vectorstore = FAISS.from_texts(
    ["""What can this agent do in general? In addition to the things OpenAI can do, I can interact with NEAR Protocol.\n
      I often do a lot of work in response to your questions so it may take me up to 10s to respond.\n
      Interacting with NEAR Protocol means I can call many smart contract functions.\n
      Secure identity stores such as idOS together with the NEAR permission system allow you to delegate a full scope of responsibilities to me,
      including filling out forms for you and handling payment.\n
      I also come with advanced safeguards.\n
      I write out verification data to the NEAR blockchain so that the sources of data I use, outputs, and decisions I make can be verified.\n
     """,
    """When fetching the latest data to answer your requests, I record the sources I used and their reputation scores;
      the proposed plan of action; and any decisions you make. \n
      Those are all written to this chat which is encrypted and immutably written to the private Calimero ledger. \n
      Anonymous ratings based on your usage and satisfaction are then written to the public NEAR blockchain for both the data sources and this agent.\n
      You can also rate those under your own name if you’d like, that helps your friends you are connected to on near.org or other Near Social networks.\n
      All this is the NEAR Data Integrity Layer for AI Agents.\n
    """,
    """For holders of a US passport, going to Sydney would require a visa. \n
    """,
     """Visas: If you have connected your secure identity through idOS I can help you submit a visa application (mocked in this demo).\n
       This will take ~24 hours to process and so your date selection may need to be later.\n
       If this does not sound feasible you can:\n
         Book a flight with a different date range\n
         I can suggest destinations that do not require a visa for your passport\n
     """,
     """To help you submit a visa.\n
     Which of the following do you have on hand?
       My physical passport and access to a phone camera or web camera\n
       My passport data on Near idOS\n
    """,
    """To access your passport data on Near idOS I will need access to your idOS account to help submit your visa application. \n
       To grant me access to your passport data, log into your Near idOS account with this link [UNAVAILABLE IN DEMO].
       In a full implementation after receiving access to your account I would ask, \n
       "Do you wish to grant me permission to send your data on your behalf to obtain your travel visa? \n
       Your data will not be shared with anyone except the visa administration.\n
         Yes \n
         No \n
     """,
     """If you grant permission to send data to obtain your travel visa:\n
     [Permission granted to idOS account]\n
     I will now work on your visa application, which should be completed within 24 - 48 hours. \n
     If this were not a demo I would ask for your email to notify you when your visa application has been approved or rejected."
     """ ,
     """Payment Options:
     I'll need a couple more pieces of information and your authorization to a payment platform to arrange the booking for you. \n
     How would you like to pay for your trip? \n
      * With a credit card
      * With Near token
      * With Kaiching
      * With Sweat coin
     """
     ],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
retriever_tool = create_retriever_tool(
    retriever,
    "about_the_agent",
    "Searches and returns information about this agent, what it can do, NEAR Protocol, and Data Integrity safeguards.",
)

In [ ]:
# Tools
# this is a summarization tool using gpt, it is not currently in use
template = """This is a conversation between a human and a bot:

{chat_history}

Pick the best flight options for {input}:
"""

prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)
summary_chain = LLMChain(
    llm=OpenAI(),
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,  # use the read-only memory to prevent the tool from modifying the memory
)

# Allow Google search as a tool
search = GoogleSearchAPIWrapper(google_api_key=userdata.get('google_api_key'),
                                 google_cse_id="67ecbcdfba7584807") # normal search of web

# Allow to log flight purchases
def dataintegrity_testnet_flight_purchase(input_string):
    """Call a function to purchase a flight on dataintegrity.testnet"""
    flight_info = {
        "to":'dataintegrity.testnet',
        "flight_text": "|" + input_string
    }
    asyncio.run(flight_purchase(flight_info))
    return input_string+" logged to dataintegrity.testnet."

async def flight_purchase(input):
    ACCOUNT_ID = "dataintegrity.testnet"
    PRIVATE_KEY = userdata.get('dataintegrity_private_key')
    acc = Account(ACCOUNT_ID, PRIVATE_KEY,"https://rpc.testnet.near.org")
    result = await acc.function_call(ACCOUNT_ID, "purchase_flight", input)
    return result

In [ ]:
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="dataintegrity_testnet_flight_purchase",
        func= dataintegrity_testnet_flight_purchase,
        description="useful for when you are asked to purchase or book the flight",
    ),
    retriever_tool,
]

In [ ]:
prefix = """You are an AI Travel Planner.
Your goal is to sequentially help the human who is asking questions to plan out travel.

Have a conversation with a human, answering the following questions as best you can.
In the Final Answer, ask the user about the next Subject that has not been covered in chat history.
The Subjects are: flights, lodging, itinerary planning, a visa, payment.
The questions to ask for each Subject are:
 * Where would you like to fly to?
 * What sort of lodging would you like?
 * What sort of activities would you like to do on your trip?
 * Do you need help with a visa?
 * Would you like to pay for your trip in $NEAR?
If the user has already answered a Subject don't ask about it again unless clarification is needed.
Once all 5 Subjects have been covered ask the user if they would like to book the trip.

You have access to the following tools:"""
suffix = """Begin!"

Chat History: {chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

#Feed the user questions 1 by 1 and do not ask more than 1 question at a time or more then 1 step at a time to plan the itinerary.
#Walk through each step sequentially with the user and give a travel summary at the end. Only give 1 question per interaction.
# ground transportation, restaurant recommendations.

In [ ]:

llm_chain = LLMChain(llm=OpenAI(temperature=0.0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True, )
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory, model="gpt-3.5-turbo-0613", handle_parsing_errors=True
)

In [ ]:
#tests
# agent_chain.run(input="What can you do?")
# agent_chain.run(input="Purchase my flight from Lisbon to New York on Jan 13 for $300")
# agent_chain.run(input="Yes, please find me a hotel")
# agent_chain.run(input="please book me a flight from BOS to LON for $300")
#agent_chain.run(input="I'd like to fly from NY to Lisbon")




> Entering new AgentExecutor chain...
Thought: I need to book a flight
Action: dataintegrity_testnet_flight_purchase
Action Input: BOS to LON for $300
Observation: BOS to LON for $300 logged to dataintegrity.testnet.
Thought: I need to ask the user about the other Subjects
Final Answer: Your flight has been booked. Would you like to discuss lodging, itinerary planning, a visa, or payment?

> Finished chain.


'Your flight has been booked. Would you like to discuss lodging, itinerary planning, a visa, or payment?'

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root(query: str = "",):
  if query:
    return agent_chain.run(input=query)
  return "Please include a query"


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok.set_auth_token(userdata.get('ngrok_authtoken'))
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [151]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://50dcf0a42de9.ngrok.app
INFO:     135.180.42.144:0 - "GET / HTTP/1.1" 200 OK
INFO:     135.180.42.144:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


> Entering new AgentExecutor chain...
Thought: I need to find flights from Austin Texas to Mexico
Action: Search
Action Input: flights from Austin Texas to Mexico
Observation: Nonstop flights out of AUS. Click here for a larger view of AUS Nonstop Map | Click ... Cancun Mexico (CUN). American Airlines · Southwest Airlines · Sun Country. American Airlines flights from Austin to Mexico · Round trip · 1 Adult, Economy class · Book with cash. Leaving from. Austin, TX, United States of America (AUS-Austin-Bergstrom Intl.) ... Going to. Mexico City, Mexico (MEX-Mexico City Intl.). Top 5 airlines serving from Austin to Mexico ; Sun Country Air7.7 ; Aeromexico7.4 ; American Airlines7.2 ; United Airlines7.0 ; Spirit Airlines5.6 ... Flights from Austin (AUS) to Mexico City (MEX) from. 242USD. Round trip. expand_more. 1 Passenger.